In [1]:
# Import packages
from nba_api.stats.endpoints import shotchartdetail
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

## Load Data 

In order to make our API request, we need player ID and team ID number to retrieve shot stats. Luckily, we can take advantage of another Github [repo](https://github.com/bttmly/nba). We can use the two JSON files via Python to load them into arrays of dictionary objects. 


In [2]:
# Load teams file
teams = json.loads(requests.get("https://raw.githubusercontent.com/bttmly/nba/master/data/teams.json").text)

# Get team ID based on team name
def get_team_id(team):
  for team in teams:
    if team['teamName'] == team:
      return team['teamId']
  return -1

In [3]:
# Load players file
players = json.loads(requests.get("https://raw.githubusercontent.com/bttmly/nba/master/data/players.json").text)


# Get player ID based on player name
def get_player_id(first, last):
  for player in players:
    if player['firstName'] == first and player['lastName'] == last:
      return player['playerId']
  return -1

In [4]:
# Get the team ID number for the Knicks
get_team_id('Golden State Warriors')


# Get the player ID number for Klay Thompson
get_player_id('Stephen', 'Curry')


201939

# Create JSON Request

In [18]:
import requests
import json

url_base = 'https://stats.nba.com/stats/shotchartdetail'

headers = {
		'Host': 'stats.nba.com',
		'Connection': 'keep-alive',
		'Accept': 'application/json, text/plain, */*',
		'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
		'Referer': 'https://stats.nba.com/',
		"x-nba-stats-origin": "stats",
		"x-nba-stats-token": "true",
		'Accept-Encoding': 'gzip, deflate, br',
		'Accept-Language': 'en-US,en;q=0.9',
	}

parameters = {
	'ContextMeasure': 'FGA',
	'LastNGames': 0,
	'LeagueID': '00',
	'Month': 0,
	'OpponentTeamID': 0,
	'Period': 0,
	'PlayerID': 201939,
	'SeasonType': 'Regular Season',
	'TeamID': 0,
	'VsDivision': '',
	'VsConference': '',
	'SeasonSegment': '',
	'Season': '2018-19',
	'RookieYear': '',
	'PlayerPosition': '',
	'Outcome': '',
	'Location': '',
	'GameSegment': '',
	'GameId': '',
	'DateTo': '',
	'DateFrom': ''
}


response = requests.get(url_base, params=parameters, headers=headers)
content = json.loads(response.content)

In [19]:
# transform contents into dataframe
results = content['resultSets'][0]
headers = results['headers']
rows = results['rowSet']
df = pd.DataFrame(rows)
df.columns = headers
df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021800002,10,201939,Stephen Curry,1610612744,Golden State Warriors,1,11,31,...,Right Side Center(RC),24+ ft.,24,226,90,1,1,20181016,GSW,OKC
1,Shot Chart Detail,0021800002,58,201939,Stephen Curry,1610612744,Golden State Warriors,1,7,57,...,Center(C),Less Than 8 ft.,6,-2,63,1,1,20181016,GSW,OKC
2,Shot Chart Detail,0021800002,131,201939,Stephen Curry,1610612744,Golden State Warriors,1,3,46,...,Center(C),Less Than 8 ft.,2,5,29,1,1,20181016,GSW,OKC
3,Shot Chart Detail,0021800002,141,201939,Stephen Curry,1610612744,Golden State Warriors,1,2,58,...,Center(C),Less Than 8 ft.,1,-6,18,1,0,20181016,GSW,OKC
4,Shot Chart Detail,0021800002,145,201939,Stephen Curry,1610612744,Golden State Warriors,1,2,49,...,Left Side(L),24+ ft.,22,-228,8,1,1,20181016,GSW,OKC


# Team Comparison 

In [16]:
from nba_api.stats.endpoints import leaguedashteamstats
import requests
import json
import pandas as pd

response = leaguedashteamstats.LeagueDashTeamStats(
    team_id_nullable='0',
    league_id_nullable='00',
    season= '2021-22',
    season_type_all_star='Regular Season',
    headers={'Accept': 'application/json, text/plain, */*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9',
'Connection': 'keep-alive',
'Host': 'stats.nba.com',
'Origin': 'https://www.nba.com',
'Referer': 'https://www.nba.com/',
'sec-ch-ua': '"Google Chrome";v="87", "\"Not;A\\Brand";v="99", "Chromium";v="87"',
'sec-ch-ua-mobile': '?1',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36',
'x-nba-stats-origin': 'stats',
'x-nba-stats-token': 'true'})


df = response.get_data_frames()[0]
df

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,1610612737,Atlanta Hawks,82,43,39,0.524,3941.0,3401,7241,0.470,...,1,22,23,10,7,6,6,14,10,Atlanta Hawks
1,1610612738,Boston Celtics,82,51,31,0.622,3981.0,3341,7167,0.466,...,13,19,2,11,5,20,12,2,10,Boston Celtics
2,1610612751,Brooklyn Nets,82,44,38,0.537,3951.0,3442,7251,0.475,...,17,24,5,21,22,16,9,15,10,Brooklyn Nets
3,1610612766,Charlotte Hornets,82,43,39,0.524,3976.0,3508,7497,0.468,...,10,5,11,12,18,18,4,16,10,Charlotte Hornets
4,1610612741,Chicago Bulls,82,46,36,0.561,3946.0,3422,7127,0.480,...,6,23,25,24,8,28,13,20,10,Chicago Bulls
5,1610612739,Cleveland Cavaliers,82,44,38,0.537,3946.0,3255,6940,0.469,...,22,26,24,15,1,11,25,13,10,Cleveland Cavaliers
6,1610612742,Dallas Mavericks,82,52,30,0.634,3951.0,3222,6982,0.461,...,3,29,28,1,14,10,24,8,10,Dallas Mavericks
7,1610612743,Denver Nuggets,82,48,34,0.585,3961.0,3416,7079,0.483,...,26,18,29,17,20,13,10,11,10,Denver Nuggets
8,1610612765,Detroit Pistons,82,23,59,0.280,3956.0,3129,7267,0.431,...,20,10,14,26,30,14,28,26,10,Detroit Pistons
9,1610612744,Golden State Warriors,82,53,29,0.646,3946.0,3323,7087,0.469,...,29,4,18,2,27,29,15,5,10,Golden State Warriors
